In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from PIL import Image, ImageDraw
print(tf.VERSION)
print(tf.keras.__version__)
import numpy as np

1.13.1
2.2.4-tf


## section 1
firstly, a list which contains weights is given. You need to load the weight into the model correctly. Then test the model on the given dataset.

In [2]:
'''
Weights_list is the parameter sets of the networks

It's structure is like:
[
[],
[layer a's weights, layer a's bias,...],
[layer b's weights]
]
'''
import pickle

    
def getParams():
    with open('params_sets', 'rb') as fp:
        weights_list = pickle.load(fp)
    weights_list = [np.array(w) for w in weights_list]
    return weights_list

In [3]:
#this is the model for the testing part
names = ['dw1', 'bn1', 'conv1', 'bn2', 'dw2', 'bn3', 'conv2', 'bn4', 'dw3', 'bn5', 'dw4', 'bn6', 'conv3', 'bn7',
        'dw5', 'bn8', 'dw6', 'bn9', 'conv4', 'bn10', 'conv5']
def model():
    rv = tf.keras.models.Sequential(
        [
        #first dw module
        layers.DepthwiseConv2D((3, 3),padding='same',depth_multiplier=1,strides=(1,1),use_bias=False),
        layers.BatchNormalization(momentum=0.1,
        epsilon=1e-5,trainable=False),
        layers.ReLU(4.0),
        layers.Conv2D(48,(1,1), padding='same',use_bias=False,strides=(1, 1)),
        layers.BatchNormalization(momentum=0.1,
        epsilon=1e-5,trainable=False),
        layers.ReLU(4.0),
        #maxpooling
        layers.MaxPool2D(strides =(2,2)),
        #second dw module
        layers.DepthwiseConv2D((3, 3),padding='same',depth_multiplier=1,strides=(1,1),use_bias=False),
        layers.BatchNormalization(momentum=0.1,
        epsilon=1e-5,trainable=False),
        layers.ReLU(4.0),
        layers.Conv2D(96,(1,1), padding='same',use_bias=False,strides=(1, 1)),
        layers.BatchNormalization(momentum=0.1,
        epsilon=1e-5,trainable=False),
        layers.ReLU(4.0),
        #maxpooling
        layers.MaxPool2D(strides =(2,2)),
        #third dw module
        layers.DepthwiseConv2D((3, 3),padding='same',depth_multiplier=1,strides=(1,1),use_bias=False),
        layers.BatchNormalization(momentum=0.1,
        epsilon=1e-5,trainable=False),
        layers.ReLU(4.0),
        layers.Conv2D(192,(1,1), padding='same',use_bias=False,strides=(1, 1)),
        layers.BatchNormalization(momentum=0.1,
        epsilon=1e-5,trainable=False),
        layers.ReLU(4.0),
        #maxpooling
        layers.MaxPool2D(strides =(2,2)),
        #fourth dw module
        layers.DepthwiseConv2D((3, 3),padding='same',depth_multiplier=1,strides=(1,1),use_bias=False),
        layers.BatchNormalization(momentum=0.1,
        epsilon=1e-5,trainable=False),
        layers.ReLU(4.0),
        layers.Conv2D(384,(1,1), padding='same',use_bias=False,strides=(1, 1)),
        layers.BatchNormalization(momentum=0.1,
        epsilon=1e-5,trainable=False),
        layers.ReLU(4.0),
        #fifth dw module
        layers.DepthwiseConv2D((3, 3),padding='same',depth_multiplier=1,strides=(1,1),use_bias=False),
        layers.BatchNormalization(momentum=0.1,
        epsilon=1e-5,trainable=False),
        layers.ReLU(4.0),
        layers.Conv2D(512,(1,1), padding='same',use_bias=False,strides=(1, 1)),
        layers.BatchNormalization(momentum=0.1,
        epsilon=1e-5,trainable=False),
        layers.ReLU(4.0),
        #output
        layers.Conv2D(10,(1,1), padding='same',use_bias=False,strides=(1, 1)),
        ]
    )
    rv.trainable = False
    return rv

def model_no_bn():
    rv = tf.keras.models.Sequential(
        [
        #first dw module
        layers.DepthwiseConv2D((3, 3),padding='same',depth_multiplier=1,strides=(1,1),use_bias=True),
        layers.ReLU(4.0),
        layers.Conv2D(48,(1,1), padding='same',use_bias=True,strides=(1, 1)),
        layers.ReLU(4.0),
        #maxpooling
        layers.MaxPool2D(strides =(2,2)),
        #second dw module
        layers.DepthwiseConv2D((3, 3),padding='same',depth_multiplier=1,strides=(1,1),use_bias=True),
        layers.ReLU(4.0),
        layers.Conv2D(96,(1,1), padding='same',use_bias=True,strides=(1, 1)),
        layers.ReLU(4.0),
        #maxpooling
        layers.MaxPool2D(strides =(2,2)),
        #third dw module
        layers.DepthwiseConv2D((3, 3),padding='same',depth_multiplier=1,strides=(1,1),use_bias=True),
        layers.ReLU(4.0),
        layers.Conv2D(192,(1,1), padding='same',use_bias=True,strides=(1, 1)),
        layers.ReLU(4.0),
        #maxpooling
        layers.MaxPool2D(strides =(2,2)),
        #fourth dw module
        layers.DepthwiseConv2D((3, 3),padding='same',depth_multiplier=1,strides=(1,1),use_bias=True),
        layers.ReLU(4.0),
        layers.Conv2D(384,(1,1), padding='same',use_bias=True,strides=(1, 1)),
        layers.ReLU(4.0),
        #fifth dw module
        layers.DepthwiseConv2D((3, 3),padding='same',depth_multiplier=1,strides=(1,1),use_bias=True),
        layers.ReLU(4.0),
        layers.Conv2D(512,(1,1), padding='same',use_bias=True,strides=(1, 1)),
        layers.ReLU(4.0),
        #output
        layers.Conv2D(10,(1,1), padding='same',use_bias=False,strides=(1, 1)),
        ]
    )
    rv.trainable = False
    return rv

In [4]:
'''
Write down the code to absorb bn layer into conv layer and maintain the same output as the original model. (please refer to HW2 Q4)
'''
def load_model_no_bn(weights_list, model_no_bn):
    model_no_bn.build(input_shape=(1,160,320,3))
    weight_idx = 1
    for layer in model_no_bn.layers:
        if layer is model_no_bn.layers[-1]:
            layer.set_weights(np.array(weights_list[weight_idx]))
            continue

        if "DepthwiseConv2D" in str(layer):
            weight = weights_list[weight_idx][0]
            bn_weights = weights_list[weight_idx + 1]
            for i in range(len(weight)):
                for j in range(len(weight[0])):
                    for k in range(len(weight[0][0])):
                        for l in range(len(weight[0][0][0])):
                            weight[i][j][k][l] = weight[i][j][k][l] * bn_weights[0][k] / np.sqrt(bn_weights[3][k])

            c = len(weight[0][0])
            bias = [0.0] * c
            for i in range(c):
                bias[i] = bn_weights[0][i] * (0 - bn_weights[2][i]) / np.sqrt(bn_weights[3][i]) + bn_weights[1][i]
            weight_idx += 2

            layer.set_weights([np.array(weight), np.array(bias)])

        elif "Conv2D" in str(layer):
            weight = weights_list[weight_idx][0]
            bn_weights = weights_list[weight_idx + 1]
            for i in range(len(weight)):
                for j in range(len(weight[0])):
                    for k in range(len(weight[0][0])):
                        for l in range(len(weight[0][0][0])):
                            weight[i][j][k][l] = weight[i][j][k][l] * bn_weights[0][l] / np.sqrt(bn_weights[3][l])

            c = len(weight[0][0][0])
            bias = [0.0] * c
            for i in range(c):
                bias[i] = bn_weights[0][i] * (0 - bn_weights[2][i]) / np.sqrt(bn_weights[3][i]) + bn_weights[1][i]
            weight_idx += 2

            layer.set_weights([np.array(weight), np.array(bias)])

'''
You should report the average IoU for each quantized model you get
'''
def do_inference(model):
    avg_iou = 0
    import json
    with open('groundtruth.txt', 'r') as outfile:
        lines = json.load(outfile)
        for line in lines[:100]:
            input_img = load_input(line[0])
            output = model.predict(input_img).transpose(0,3,1,2)
            avg_iou+= bbox_iou(get_box(output),line[1])
        avg_iou = avg_iou/len(lines[:100])
        return avg_iou
    

'''
Now finish the function to compute the iou between two given box.

You can refer to the website: https://www.pyimagesearch.com/2016/11/07/intersection-over-union-iou-for-object-detection/
'''
# box := (x, y, delta x, delta y)
def bbox_iou(boxA, boxB):
    '''your code here'''
    # determine the (x, y)-coordinates of the intersection rectangle
    boxA[2] += boxA[0]
    boxA[3] += boxA[1]
    boxB[2] += boxB[0]
    boxB[3] += boxB[1]
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
 
    # compute the area of intersection rectangle
    interArea = max(0, xB - xA) * max(0, yB - yA)
 
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
 
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
 
    # return the intersection over union value
    return iou
'''
This is the function to get the predict box (x,y,w,h)
'''
def sigmoid(x):
    return 1 / (1 + np.exp(-x))
def get_box(output):
    anchors = [1.4940052559648322, 2.3598481287086823, 4.0113013115312155, 5.760873975661669]
    h = output.shape[2]
    w = output.shape[3]
    output = output.reshape(2,5,800).transpose(1,0,2).flatten().reshape(5,1600)
    grid_x = np.tile(np.tile(np.linspace(0,w-1,w),h).reshape(h,w),(2,1,1)).flatten()
    grid_y =np.tile(np.tile(np.linspace(0,h-1,h),w).reshape(w,h).T,(2,1,1)).flatten()
    xs = sigmoid(output[0]) + grid_x
    ys = sigmoid(output[1]) + grid_y
    anchor_w = np.zeros(1600)
    anchor_h = np.zeros(1600)
    anchor_w[0:800] = anchors[0]
    anchor_w[800:1600] = anchors[2]
    anchor_h[0:800] = anchors[1]
    anchor_h[800:1600] = anchors[3]
    ws = np.exp(output[2]) * anchor_w
    hs = np.exp(output[3]) * anchor_h
    ind = np.argmax(output[4])
    bcx = xs[ind]
    bcy = ys[ind]
    bw = ws[ind]
    bh = hs[ind]
    box = [bcx/w, bcy/h, bw/w, bh/h]
    return box
'''
The function is to convert the image into the input type.
'''
def load_input(path):
    img = Image.open(path)
    img = img.resize((320,160))
    input_img = np.asarray(img).astype(np.float32)
    input_img = (input_img/255 - 0.5)/0.25
    return input_img[np.newaxis,:]

'''
write down your code to load the model
'''
def load_model(weights, in_model):
    layer_list = [("ReLU" not in ls) and ("MaxPooling" not in ls) for ls in map(str, in_model.layers)]
    in_model.build([1,160,320,3])
    weight_idx = 1
    for idx, val in enumerate(layer_list):
        if val == False:
            continue
        in_model.layers[idx].set_weights(weights[weight_idx])
        weight_idx += 1

'''
Based on the model_no_bn quantize the weights to 16 bits, 8 bits respectively.

The requirement of quantization is given below:

* For each layer's weights, set the upper bound as the minimum 2^n which is larger than the maximum value of unsigned weights. (eg: if the maximum value is 4.2375 and the minimum value is -7.83421, then the upper bound is 2^3 = 8)

* Note that for each layer, the distribution of weights could be different.

* The sign takes one bit. For example, if the upper bound is 8 and 5 bits is given for floating part, it actually takes 9 bits.

* Do not quantize the bias!

and get the accuracy report
'''
import math
def quantize_layer(weights, nbits):
    for w in weights:
        assert(type(w) != type([]))

    abs_max = np.max(np.abs(weights))
    sign_bits = 1
    exp_bits = None
    exp = 0
    exp = np.max([np.ceil(np.log2(abs_max)), 0])
    #print("Largest exp is {exp}".format(exp=exp))
    if exp == 0:
        exp_bits = 0
    else:
        exp_bits = np.ceil(np.log2(exp))
    if (exp_bits > nbits-1):
        print("Warning, {exp_bits} of {nbits} are used for exponent".format(exp_bits=exp_bits,
                                                                           nbits=nbits))
        exp_bits = min(exp_bits, nbits-1)
    mantissa_bits = nbits - sign_bits - exp_bits
    #print("Number of exp bits: {exp_bits}".format(exp_bits=exp_bits))
    if (exp_bits > 8):
        print("Exponent too big, abs max is {abs_max}".format(abs_max=abs_max))
    if (np.isnan(exp_bits) or np.isnan(mantissa_bits)):
        print("Found NaNs: (abs_max, exp_bits, mantissa_bits) = ({am}, {eb}, {mb})\n".format(
                            am=abs_max, eb=exp_bits, mb=mantissa_bits))
    
    #print(sign_bits, exp_bits, mantissa_bits)
    def callback(w):
        assert("list" not in str(type(w)))
        abs_w = abs(w)
        # mantissa evenly divides the half-open range [2**exp, 2**(exp+1))
        exp = np.floor(np.log2(abs_w))
        w_exp = 2.0**exp
        mantissa_interval = 2.0**exp / (2.0**mantissa_bits)
        w_mant = abs_w - w_exp
        w_mant = (mantissa_interval) * np.round(w_mant / mantissa_interval)
        rv = (w_exp + w_mant) * np.sign(w)
        #print(rv)
        if np.isnan(rv).any():
            print("Found nan {w}: exp_bits: {eb}, mantissa_bits: {mb}".format(
            w=w, eb=exp_bits, mb=mantissa_bits))
        return rv
        
    if "ndarray" in str(type(weights)):
        return np.array(map(callback, weights))
    else:
        print("Expected numpy array")
        return None
    #return [callback(w) for w in weights]

def quantize_model_raw(weights, nbits):    
    for idx, wlist in enumerate(weights):
        if len(wlist) == 0:
            continue
        if type(wlist[0]) == type([]):
            wlist = quantize_model(wlist)
        else:
            weights[idx] = quantize_layer(wlist, nbits)        
    return weights


def quantize_model(model, nbits):
    for layer in model.layers:
        if "Conv" not in str(layer):
            continue
        new_weights = [quantize_layer(m, nbits) for m in layer.get_weights()]
        layer.set_weights(new_weights)
    return model, new_weights

    

In [57]:
n = np.array([1.2, 2.1, -0.23, 1.242])
print(quantize_layer(n, 32))
n = np.array([-0.23, 0.75])
print(quantize_layer(n, 4))
m = model()
load_model(getParams(), m)

[ 1.2    2.1   -0.23   1.242]
[-0.234375  0.75    ]


In [58]:
quantize_model(m, 16)

(<tensorflow.python.keras.engine.sequential.Sequential at 0x7f9eac702490>,
 [array([[[[-0.02560806,  0.00185621,  0.02195549, ..., -0.00584257,
             0.00628638, -0.36338806],
           [-0.01747561,  0.02075624, -0.00821996, ..., -0.00965595,
             0.00861645, -0.04374599],
           [ 0.01971388,  0.00125912, -0.03931427, ..., -0.05925083,
             0.07988739, -0.01765728],
           ...,
           [-0.0065037 , -0.03998756, -0.05000877, ...,  0.015347  ,
            -0.01759577, -0.11148834],
           [-0.01250672,  0.01442003,  0.04115009, ..., -0.00356311,
            -0.06481743,  0.1586647 ],
           [ 0.00347739,  0.02072859,  0.04285908, ..., -0.02249575,
             0.01962471,  0.0037328 ]]]], dtype=float32)])

In [46]:
mnb2 = model_no_bn()
load_model_no_bn(getParams(), mnb2)
mnb2, _ = quantize_model(mnb2, 32)
mnb1 = model_no_bn()
load_model_no_bn(getParams(), mnb1)

In [47]:
#mnb1 = model_no_bn()
#mnb2 = model_no_bn()
#load_model_no_bn(getParams(), mnb1)
#load_model_no_bn(getParams(), mnb2)
#_, w1 = quantize_model(mnb1, 8)
#mnb2, _ = quantize_model(mnb2, 32)

for idx in range(len(mnb2.layers)):
    print(mnb2.layers[idx])
    for widx in range(len(mnb2.layers[idx].get_weights())):
        print(np.mean(np.abs(mnb2.layers[idx].get_weights()[widx] - mnb1.layers[idx].get_weights()[widx])))

#w1 = None
#w2 = None
#del mnb1
#del mnb2

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [5]:
mnb = model_no_bn()
load_model_no_bn(getParams(), mnb)
m = model()
load_model(getParams(), m)

Instructions for updating:
Colocations handled automatically by placer.


In [14]:
from matplotlib.pyplot import imshow
def drawBox(image_path, bbox, write_path):
    WIDTH=640.0
    HEIGHT=360.0
    src_img = Image.open(image_path)
    coords = (((bbox[0]-bbox[2]/2)*WIDTH, (bbox[1]-bbox[3]/2)*HEIGHT), ((bbox[0]+bbox[2]/2)*WIDTH, (bbox[1]+bbox[3]/2)*HEIGHT))
    draw = ImageDraw.Draw(src_img)
    draw.rectangle(coords)
    src_img.save(write_path)
    
img_path = 'images/2.jpg'
input_img = load_input(img_path)
#output = m.predict(input_img).transpose(0,3,1,2)
#bbox = get_box(output)
bbox = [0.8880645155906677, 0.6772263944149017, 0.02124013871572325, 0.058586649582813566]
print(bbox)
drawBox(img_path, bbox)
#mnb = None

[0.8880645155906677, 0.6772263944149017, 0.02124013871572325, 0.058586649582813566]


TypeError: drawBox() takes exactly 3 arguments (2 given)

In [49]:
'''
This is the cell to test your weights correctness.

The output should be :
[0.8880645155906677, 0.6772263944149017, 0.02124013871572325, 0.058586649582813566]
'''
input_img = load_input('images/2.jpg')
m = model()
load_model(getParams(), m)
output = m.predict(input_img).transpose(0,3,1,2)
print (get_box(output))
m = None

[0.8880645155906677, 0.6772263944149017, 0.021240140941964566, 0.05858664254991809]


In [50]:
'''
The iou should be about 67%
'''
mnb = model_no_bn()
load_model_no_bn(getParams(), mnb)
m = model()
load_model(getParams(), m)
print(do_inference(m))
print(do_inference(mnb))
m = None
mnb = None

0.6385218143190471
0.6394115229971922


## section 2
In this section, you need to convert the model into a model without batch normalization layers. The output of two model should be the same. Then you are required to quantize the model without batch normalization.

In [17]:
'''
Based on the model_no_bn

Now you can quantize both weights and bias parts.

Explore eight different combination of weights and parts and specify your methods' details, and get the accuracy report
'''
input_img = load_input('images/2.jpg')
img_path = 'images/2.jpg'

m = model()
mnb = model_no_bn()
for nbits in [8, 12, 16, 20, 24, 28, 32]:
    load_model(getParams(), m)
    quantize_model(m, nbits)
    bn = do_inference(m)
    load_model_no_bn(getParams(), mnb)
    quantize_model(mnb, nbits)
    no_bn = do_inference(mnb)
    output = m.predict(input_img).transpose(0,3,1,2)
    output_no_bn = mnb.predict(input_img).transpose(0,3,1,2)

    print("With {nbits} quantized floats:\n\tBN: {bn}\n\tNo BN: {no_bn}".format(
            nbits=nbits, bn=bn, no_bn=no_bn))
    print(get_box(output))
    print(get_box(output_no_bn))
    write_path = "output/mnb_{nbits}.jpg".format(nbits=str(nbits))
    drawBox(img_path, get_box(output_no_bn), write_path)
    write_path = "output/m_{nbits}.jpg".format(nbits=str(nbits))
    drawBox(img_path, get_box(output), write_path)
    
    
m = None
mnb = None

With 8 quantized floats:
	BN: 0.637164661016
	No BN: 0.508875281964
[0.888053260743618, 0.6771463364362716, 0.021262824114716566, 0.0588954534736361]
[0.8868082776665688, 0.6787367999553681, 0.016961657995015356, 0.06047556574731]
With 12 quantized floats:
	BN: 0.638487680878
	No BN: 0.640184099694
[0.888065131008625, 0.6772283524274826, 0.021239384019917686, 0.058580302394650285]
[0.8882624104619026, 0.6777048885822297, 0.022280332159926867, 0.05455664927023237]
With 16 quantized floats:
	BN: 0.638521388576
	No BN: 0.637607516674
[0.8880644798278808, 0.6772268295288086, 0.02124017878806691, 0.058587535727643004]
[0.8883731961250305, 0.6777315616607666, 0.022121961805306012, 0.05542029586706213]
With 20 quantized floats:
	BN: 0.638521851425
	No BN: 0.639412643879
[0.8880645155906677, 0.6772263914346695, 0.021240132036999308, 0.05858666013215678]
[0.8883777961134911, 0.6777168184518814, 0.022150776046637757, 0.05548471367313588]
With 24 quantized floats:
	BN: 0.638521814319
	No BN: 0.63